# 5. LME銅3M先物 - トレーディング戦略分析

## 概要
LME銅3M先物の実践的なトレーディング戦略を開発・バックテストします。

### 実装戦略
1. 移動平均クロスオーバー戦略
2. ミーンリバージョン戦略
3. モメンタム戦略（RSIベース）
4. ボラティリティブレイクアウト戦略
5. 機械学習ベース戦略

### パフォーマンス指標
- 総リターン
- シャープレシオ
- 最大ドローダウン
- 勝率
- VaR・カルマーレシオ

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# データベース接続
engine = create_engine('postgresql://Yusuke@localhost:5432/lme_copper_db')

print("=== LME銅3M先物トレーディング戦略分析システム ===")
print("データベース接続完了")

## 1. データ読み込みと前処理

In [ ]:
# LME銅3M先物データの読み込み
query = """
SELECT 
    date,
    m3_price as price,
    m3_volume as volume,
    m1_price,
    m2_price,
    m4_price,
    m5_price,
    m6_price
FROM lme_copper_futures
WHERE m3_price IS NOT NULL
ORDER BY date
"""

df = pd.read_sql_query(query, engine)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)

# 基本的なテクニカル指標を計算
df['returns'] = df['price'].pct_change()

# 移動平均
df['ma5'] = df['price'].rolling(window=5).mean()
df['ma10'] = df['price'].rolling(window=10).mean()
df['ma20'] = df['price'].rolling(window=20).mean()
df['ma50'] = df['price'].rolling(window=50).mean()

print(f"データ期間: {df.index.min()} - {df.index.max()}")
print(f"総レコード数: {len(df):,}")

# 欠損値除去
df = df.dropna()

print(f"欠損値除去後: {len(df):,}件")